In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
109,this isn't 'Bonnie and Clyde' or 'Thelma and L...,positive
155,I'm working my way through the Horror Classics...,negative
53,Perhaps once in a generation a film comes alon...,positive
106,Susan Slept Here turned out to be Dick Powell'...,negative
797,There is nothing unique in either the TV Serie...,negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
109,bonnie clyde thelma louise fine road movie set...,positive
155,working way horror classic movie pack collecti...,negative
53,perhaps generation film come along perfection ...,positive
106,susan slept turned dick powell s swan song per...,negative
797,nothing unique either tv series movie prequel ...,negative


In [5]:
df['sentiment'].value_counts()

sentiment
negative    274
positive    226
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
109,bonnie clyde thelma louise fine road movie set...,1
155,working way horror classic movie pack collecti...,0
53,perhaps generation film come along perfection ...,1
106,susan slept turned dick powell s swan song per...,0
797,nothing unique either tv series movie prequel ...,0


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [13]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [15]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/psaurav170/Mlops-Capstone-Project.mlflow')
dagshub.init(repo_owner='psaurav170', repo_name='Mlops-Capstone-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-12-24 00:17:26,077 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/psaurav170/Mlops-Capstone-Project "HTTP/1.1 200 OK"


Initialized MLflow to track repo "psaurav170/Mlops-Capstone-Project"

2025-12-24 00:17:26,088 - INFO - Initialized MLflow to track repo "psaurav170/Mlops-Capstone-Project"


Repository psaurav170/Mlops-Capstone-Project initialized!

2025-12-24 00:17:26,093 - INFO - Repository psaurav170/Mlops-Capstone-Project initialized!


<Experiment: artifact_location='mlflow-artifacts:/91f1575c222542d7814786fe5d8db790', creation_time=1766515246550, experiment_id='0', last_update_time=1766515246550, lifecycle_stage='active', name='Logistic Regression Baseline', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [16]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-12-24 00:17:33,992 - INFO - Starting MLflow run...
2025-12-24 00:17:34,397 - INFO - Logging preprocessing parameters...
2025-12-24 00:17:35,836 - INFO - Initializing Logistic Regression model...
2025-12-24 00:17:35,841 - INFO - Fitting the model...
2025-12-24 00:17:35,876 - INFO - Model training complete.
2025-12-24 00:17:35,877 - INFO - Logging model parameters...
2025-12-24 00:17:36,316 - INFO - Making predictions...
2025-12-24 00:17:36,319 - INFO - Calculating evaluation metrics...
2025-12-24 00:17:36,345 - INFO - Logging evaluation metrics...
2025-12-24 00:17:38,256 - INFO - Saving and logging the model...
2025/12/24 00:17:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-12-24 00:17:58,882 - INFO - Model training and logging completed in 24.48 seconds.
2025-12-24 00:17:58,884 - INFO - Accuracy: 0.66
2025-12-24 00:17:58,888 - INFO - Precision: 0.625
2025-12-24 00:17:58,892 - INFO - Recall: 0.6521739130434783
2025-12-24 00:17:58,896 

🏃 View run sassy-goat-748 at: https://dagshub.com/psaurav170/Mlops-Capstone-Project.mlflow/#/experiments/0/runs/4d39522d1b354c0cac625701e5b64b74
🧪 View experiment at: https://dagshub.com/psaurav170/Mlops-Capstone-Project.mlflow/#/experiments/0
